# Stock Price Prediction

## Data Preprocessing

### Importing the Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Importing the Dataset

In [ ]:
dataset_train = pd.read_csv('data/data_train.csv')
training_set = dataset_train.iloc[:, 1:2].values

### Feature Scaling

In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

### Creating Data Structures

In [ ]:
X_train = []
y_train = []

for i in range(60, 506):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

### Reshaping

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) #type: ignore

## Building the Architecture of the Recurrent Neural Network

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=50, return_sequences=False))
regressor.add(Dropout(rate=0.2))
regressor.add(Dense(units=1))

In [ ]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

### Training the RNN

In [ ]:
regressor.fit(x=X_train, y=y_train, epochs=150, batch_size=32)

## Evaluating the Model

In [ ]:
dataset_test = pd.read_csv('data/data_test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis=0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1, 1) #type: ignore
inputs = sc.transform(inputs)

X_test = []

for i in range(60, 506):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predictd_stock_price = regressor.predict(X_test)
predictd_stock_price = sc.inverse_transform(predictd_stock_price)

### Visualizing the Results

In [ ]:
plt.plot(real_stock_price, color='red', label='Real Netflix Stock Price')
plt.plot(predictd_stock_price, color='blue', label='Model')
plt.title('Netflix Stock Price Predictions')
plt.xlabel('Time')
plt.ylabel('Netflix Stock Price')
plt.legend()
plt.show()